# Returning Structured Output | 構造化された出力の返却

このノートブックでは、エージェントが構造化された出力を返す方法について説明しています。デフォルトでは、ほとんどのエージェントは単一の文字列を返すことが多いです。しかし、エージェントにより構造化されたものを返してもらうことが有益な場合もしばしばあります。

> This notebook covers how to have an agent return a structured output.
> By default, most of the agents return a single string.
> It can often be useful to have an agent return something with more structure.

これの良い例は、いくつかの情報源をもとに質問応答を行うエージェントです。例えば、私たちはエージェントに答えだけでなく、使用した情報源のリストも提供してほしいと考えています。そして、私たちは出力が大まかに以下のスキーマに従うことを望んでいます：

> A good example of this is an agent tasked with doing question-answering over some sources.
> Let's say we want the agent to respond not only with the answer, but also a list of the sources used.
> We then want our output to roughly follow the schema below:

```python
class Response(BaseModel):
    """Final response to the question being asked"""
    answer: str = Field(description = "The final answer to respond to the user")
    sources: List[int] = Field(description="List of page chunks that contain answer to the question. Only include a page chunk if it contains relevant information")
```

このノートブックでは、リトリバー（検索）ツールを持ち、正しいフォーマットで応答するエージェントについて解説します。

> In this notebook we will go over an agent that has a retriever tool and responds in the correct format.




## Create the Retriever | リトリーバーの作成

このセクションでは、「一般教書演説」を含む模擬データを用いて、情報を取得するシステムを設定する作業を行います。特に、各ドキュメントのメタデータに「page\_chunk」というタグを追加することが重要です。これは、情報源フィールドを模擬するための架空のデータに過ぎません。実際の運用では、このタグはドキュメントのURLやパスであることが一般的です。

> In this section we will do some setup work to create our retriever over some mock data containing the "State of the Union" address. Importantly, we will add a "page\_chunk" tag to the metadata of each document. This is just some fake data intended to simulate a source field. In practice, this would more likely be the URL or path of a document.




In [1]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [16]:
# Load in document to retrieve over
loader = TextLoader("../../state_of_the_union.txt")
documents = loader.load()

# Split document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Here is where we add in the fake source information
for i, doc in enumerate(texts):
    doc.metadata["page_chunk"] = i

# Create our retriever
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")
retriever = vectorstore.as_retriever()

## Create the tools | ツールを作成する

これからエージェントに提供するツールを作成しましょう。この場合、それは一つだけです - 私たちのリトリバーを包み込むツールです。

> We will now create the tools we want to give to the agent. In this case, it is just one - a tool that wraps our retriever.




In [4]:
from langchain.agents.agent_toolkits.conversational_retrieval.tool import (
    create_retriever_tool,
)

retriever_tool = create_retriever_tool(
    retriever,
    "state-of-union-retriever",
    "Query a retriever to get information about state of the union address",
)

## Create response schema | レスポンススキーマを作成する

ここではレスポンススキーマを定義します。この場合、最終的な回答には`answer`のためのフィールドと、`sources`のリストのためのもう一つのフィールドが含まれることを望んでいます。

> Here is where we will define the response schema. In this case, we want the final answer to have two fields: one for the `answer`, and then another that is a list of `sources`




In [5]:
from typing import List

from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from pydantic import BaseModel, Field


class Response(BaseModel):
    """Final response to the question being asked"""

    answer: str = Field(description="The final answer to respond to the user")
    sources: List[int] = Field(
        description="List of page chunks that contain answer to the question. Only include a page chunk if it contains relevant information"
    )

## Create the custom parsing logic | カスタム解析ロジックを作成する

ここでは、カスタムの解析ロジックを作成します。これがどのように機能するかというと、`Response` スキーマを OpenAI LLM に `functions` パラメータを介して渡すことによります。これは、エージェントに使用させるツールを渡す方法に似ています。

> We now create some custom parsing logic.
> How this works is that we will pass the `Response` schema to the OpenAI LLM via their `functions` parameter.
> This is similar to how we pass tools for the agent to use.

OpenAIによって`Response`関数が呼び出されたとき、それをユーザーへの返信のシグナルとして使用します。OpenAIによって他のいかなる関数が呼び出された場合も、それをツールの起動として扱います。

> When the `Response` function is called by OpenAI, we want to use that as a signal to return to the user.
> When any other function is called by OpenAI, we treat that as a tool invocation.

したがって、私たちの解析ロジックには以下のブロックがあります：

> Therefore, our parsing logic has the following blocks:

* 関数が呼び出されない場合は、ユーザーに応答するためにレスポンスを使用すると想定し、そのため`AgentFinish`を返すことになります。

  > If no function is called, assume that we should use the response to respond to the user, and therefore return `AgentFinish`

* `Response` 関数が呼び出された場合、その関数に対する入力（私たちの構造化された出力）を用いてユーザーに応答し、その結果として `AgentFinish` を返すようにしてください。

  > If the `Response` function is called, respond to the user with the inputs to that function (our structured output), and therefore return `AgentFinish`

* 他の関数が呼び出された場合は、それをツールの起動とみなし、したがって`AgentActionMessageLog`を返してください。

  > If any other function is called, treat that as a tool invocation, and therefore return `AgentActionMessageLog`


`AgentAction`ではなく`AgentActionMessageLog`を使用している点に注意してください。これにより、将来エージェントのプロンプトに渡すために使用できるメッセージのログを添付できます。

> Note that we are using `AgentActionMessageLog` rather than `AgentAction` because it lets us attach a log of messages that we can use in the future to pass back into the agent prompt.




In [6]:
import json

from langchain_core.agents import AgentActionMessageLog, AgentFinish

In [17]:
def parse(output):
    # If no function was invoked, return to user
    if "function_call" not in output.additional_kwargs:
        return AgentFinish(return_values={"output": output.content}, log=output.content)

    # Parse out the function call
    function_call = output.additional_kwargs["function_call"]
    name = function_call["name"]
    inputs = json.loads(function_call["arguments"])

    # If the Response function was invoked, return to the user with the function inputs
    if name == "Response":
        return AgentFinish(return_values=inputs, log=str(function_call))
    # Otherwise, return an agent action
    else:
        return AgentActionMessageLog(
            tool=name, tool_input=inputs, log="", message_log=[output]
        )

## Create the Agent | エージェントを作成する

これで全てを一つにまとめることができます！このエージェントの構成要素は以下の通りです：

> We can now put this all together! The components of this agent are:

* プロンプト：ユーザーの質問と`agent_scratchpad`（中間ステップ）のためのプレースホルダーを含むシンプルなプロンプト

  > prompt: a simple prompt with placeholders for the user's question and then the `agent_scratchpad` (any intermediate steps)

* ツール：LLMにツールと`Response`フォーマットを関数として添付できます

  > tools: we can attach the tools and `Response` format to the LLM as functions

* `agent_scratchpad`の中間ステップをフォーマットするために、標準の`format_to_openai_function_messages`を使用します。この関数は中間ステップを取り、それらをAIMessagesとFunctionMessagesとしてフォーマットします。

  > format scratchpad: in order to format the `agent_scratchpad` from intermediate steps, we will use the standard `format_to_openai_function_messages`. This takes intermediate steps and formats them as AIMessages and FunctionMessages.

* 出力パーサー：LLMの応答を解析するために、上記で説明したカスタムパーサーを使用します

  > output parser: we will use our custom parser above to parse the response of the LLM

* AgentExecutor: 標準のAgentExecutorを使用して、エージェント-ツール-エージェント-ツールのループを実行します...

  > AgentExecutor: we will use the standard AgentExecutor to run the loop of agent-tool-agent-tool...





In [ ]:
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [9]:
llm = ChatOpenAI(temperature=0)

In [10]:
llm_with_tools = llm.bind(
    functions=[
        # The retriever tool
        format_tool_to_openai_function(retriever_tool),
        # Response schema
        convert_pydantic_to_openai_function(Response),
    ]
)

In [11]:
agent = (
    {
        "input": lambda x: x["input"],
        # Format agent scratchpad from intermediate steps
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | parse
)

In [14]:
agent_executor = AgentExecutor(tools=[retriever_tool], agent=agent, verbose=True)

## Run the agent | エージェントを実行する

これでエージェントを実行できます！`answer`と`sources`の2つのキーを含む辞書で応答する様子に注目してください。

> We can now run the agent! Notice how it responds with a dictionary with two keys: `answer` and `sources`




In [18]:
agent_executor.invoke(
    {"input": "what did the president say about kentaji brown jackson"},
    return_only_outputs=True,
)



> Entering new AgentExecutor chain...
[Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'page_chunk': 31, 'source': '../../state_of_the_union.txt'}), Document(page_content='One was stationed at bases and breathing in t

{'answer': "President mentioned Ketanji Brown Jackson as a nominee for the United States Supreme Court and praised her as one of the nation's top legal minds.",
 'sources': [31]}